In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [21]:
!git clone https://github.com/ntasfi/PyGame-Learning-Environment

Cloning into 'PyGame-Learning-Environment'...
remote: Enumerating objects: 1118, done.
remote: Total 1118 (delta 0), reused 0 (delta 0), pack-reused 1118
Receiving objects: 100% (1118/1118), 8.06 MiB | 17.85 MiB/s, done.
Resolving deltas: 100% (592/592), done.


In [50]:
import gym
from gym import spaces
import numpy as np
from ple import PLE
from ple.games.flappybird import FlappyBird

class FlappyBirdEnv(gym.Env):
    def __init__(self):
        self.game = FlappyBird()
        self.p = PLE(self.game, fps=30, display_screen=True)
        self.p.init()
        self.action_set = self.p.getActionSet()

        # Define observation space using the keys from getGameState
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(8,), dtype=np.float32)
        self.action_space = spaces.Discrete(len(self.action_set))

    def reset(self):
        self.p.reset_game()
        return self.get_observation()

    def step(self, action):
        # Ensure action is within valid range
        if action >= len(self.action_set):
            action = 0
        reward = self.p.act(self.action_set[action])
        state = self.get_observation()
        done = self.p.game_over()
        return state, reward, done, {}

    def get_observation(self):
        state = self.p.getGameState()
        return np.array(list(state.values()), dtype=np.float32)

    def render(self, mode='rgb_array'):
        if mode == 'rgb_array':
            return self.p.getScreenRGB()
        elif mode == 'human':
            self.p.display_screen = True
        else:
            super(FlappyBirdEnv, self).render(mode=mode)

    def close(self):
        self.p.close()

# Register the custom environment
gym.envs.registration.register(id='FlappyBird-v0', entry_point=FlappyBirdEnv)


  logger.warn(f"Overriding environment {spec.id}")



In [42]:
import os
import neat
import pickle
import numpy as np
import gym

# Define the evaluation function for the NEAT algorithm
def eval_genomes(genomes, config):
    env = gym.make('FlappyBird-v0')
    for genome_id, genome in genomes:
        observation = env.reset()
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        genome.fitness = 0
        done = False
        while not done:
            action = np.argmax(net.activate(observation))
            observation, reward, done, info = env.step(action)
            genome.fitness += reward
    env.reset()

# Set up the NEAT configuration
def run_neat(config_file):
    config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_file)
    p = neat.Population(config)
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    winner = p.run(eval_genomes, 50)

    with open('winner.pkl', 'wb') as f:
        pickle.dump(winner, f)

    return winner

# Load the NEAT configuration and run the algorithm
config_path = 'config-feedforward.txt'
winner = run_neat(config_path)



 ****** Running generation 0 ****** 

Population's average fitness: -5.00000 stdev: 0.00000
Best fitness: -5.00000 - size: (3, 0) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.098, standard deviation 0.289
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    0    50     -5.0    0.000     0
Total extinctions: 0
Generation time: 6.331 sec

 ****** Running generation 1 ****** 

Population's average fitness: -5.00000 stdev: 0.00000
Best fitness: -5.00000 - size: (3, 0) - species 1 - id 1
Average adjusted fitness: 0.000
Mean genetic distance 1.326, standard deviation 0.291
Population of 50 members in 1 species:
   ID   age  size  fitness  adj fit  stag
  ====  ===  ====  =======  =======  ====
     1    1    50     -5.0    0.000     1
Total extinctions: 0
Generation time: 5.026 sec (5.678 average)

 ****** Running generation 2 ****** 

Population's average fitness: -5.00000 stdev: 0

In [51]:
import gym
import moviepy.editor as mpy
import neat
import pickle
import numpy as np

def visualize_winner(winner, config):
    env = gym.make('FlappyBird-v0')
    observation = env.reset()
    net = neat.nn.FeedForwardNetwork.create(winner, config)
    frames = []
    done = False

    while not done:
        frame = env.render()
        if frame is not None:
            frames.append(frame)
        action = np.argmax(net.activate(observation))
        observation, reward, done, _ = env.step(action)

    env.reset()  # Reset the environment instead of closing

    # Ensure there are no None frames in the list
    if len(frames) == 0:
        print("No frames were captured.")
        return

    frames = [frame for frame in frames if frame is not None]

    # Create a video from the frames
    clip = mpy.ImageSequenceClip(frames, fps=30)
    clip.write_videofile('flappy_bird_neat.mp4')

# Load the trained model and visualize
with open('winner.pkl', 'rb') as f:
    winner = pickle.load(f)

config = neat.config.Config(neat.DefaultGenome, neat.DefaultReproduction, neat.DefaultSpeciesSet, neat.DefaultStagnation, config_path)
visualize_winner(winner, config)


Moviepy - Building video flappy_bird_neat.mp4.
Moviepy - Writing video flappy_bird_neat.mp4



Moviepy - Done !
Moviepy - video ready flappy_bird_neat.mp4
